## Instructions

1. Follow these instructions from `packages/grid/veilid/development.md` to build veilid docker containers:
   ```bash
   cd packages/grid/veilid && docker build -f veilid.dockerfile -t veilid:0.1 .
   ```
2. From within the `packages/grid/veilid` directory run the receiver docker container on port 4000:
   ```bash
   docker run -it -e DEV_MODE=True -p 4000:4000 -v $(pwd)/server:/app/server veilid:0.1
   ```
3. On a separate terminal tab/window, cd into `packages/grid/veilid` directory again and run the sender docker container on port 4001:
   ```bash
   docker run -it -e DEV_MODE=True -p 4001:4000 -v $(pwd)/server:/app/server veilid:0.1
   ```
4. Follow and run the below cells to test out sending large messages through Veilid. You may also use the **`Run All`** notebook function once the above two docker containers are up and running.

### 1. Set up imports

In [ ]:
# stdlib
import json
import logging
from pprint import pprint
import random
import time

# third party
import requests

logging.basicConfig(level=logging.INFO, format="%(message)s")

### 2. Set up receiver

In [ ]:
RECEIVER_HOST = "localhost"
RECEIVER_PORT = 4000
RECEIVER_BASE_ADDRESS = f"http://{RECEIVER_HOST}:{RECEIVER_PORT}"

requests.post(f"{RECEIVER_BASE_ADDRESS}/generate_vld_key")
res = requests.get(f"{RECEIVER_BASE_ADDRESS}/retrieve_vld_key")
receiver_vld_key = res.json()["message"]
logging.info(f"{'=' * 30}\n{receiver_vld_key}\n{'=' * 30}")

### 3. Set up sender

In [ ]:
SENDER_HOST = "localhost"
SENDER_PORT = 4001
SENDER_BASE_ADDRESS = f"http://{SENDER_HOST}:{SENDER_PORT}"

requests.post(f"{SENDER_BASE_ADDRESS}/generate_vld_key")
res = requests.get(f"{SENDER_BASE_ADDRESS}/retrieve_vld_key")
sender_vld_key = res.json()["message"]
logging.info(f"{'=' * 30}\n{sender_vld_key}\n{'=' * 30}")

### 4. Declare utility functions

In [ ]:
def send_test_request(request_size_bytes, response_size_bytes):
    """
    Send a test request of the specified size and receive a response of.

    Args:
        request_size_bytes (int): Size of the request body in bytes.
        response_size_bytes (int): Expected size of the response body in bytes.

    Returns:
        tuple: A tuple containing the total transfer size, total time taken and success status.
    """
    message = build_vld_message(request_size_bytes, response_size_bytes)
    json_data = {
        "vld_key": receiver_vld_key,
        "message": message,
    }

    logging.info(f"Sending message of size {len(message) // 1024} KB...")

    start = time.time()
    app_call = requests.post(f"{SENDER_BASE_ADDRESS}/app_call", json=json_data)
    end = time.time()

    response = app_call.content
    response_len = len(response)
    response = response.decode()
    response_pretty = (
        response if len(response) <= 100 else f"{response[:50]}...{response[-50:]}"
    )

    total_xfer = request_size_bytes + response_size_bytes
    total_time = round(end - start, 2)

    success = "received_request_body_length" in response
    logging.info(f"[{total_time}s] Response({response_len} B): {response_pretty}")
    return total_xfer, total_time, success


def build_vld_message(request_size_bytes, response_size_bytes):
    """
    Build a message of length `request_size_bytes`. Padded with random characters.

    Args:
        request_size_bytes (int): Size of the request body in bytes.
        response_size_bytes (int): Expected size of the response body in bytes.

    Returns:
        dict: The constructed request body.
    """
    endpoint = f"{RECEIVER_BASE_ADDRESS}/test_veilid_streamer"
    message = {
        "method": "POST",
        "url": endpoint,
        "json": {
            "expected_response_length": response_size_bytes,
            "random_padding": "",
        },
    }
    padding_length = request_size_bytes - len(json.dumps(message))
    random_padding = generate_random_alphabets(padding_length)
    message["json"]["random_padding"] = random_padding
    return json.dumps(message)


def generate_random_alphabets(length):
    return "".join([random.choice("abcdefghijklmnopqrstuvwxyz") for _ in range(length)])


def bytes_to_human_readable(size_in_bytes):
    if size_in_bytes >= (2**20):
        size_in_mb = size_in_bytes / (2**20)
        return f"{size_in_mb:.2f} MB"
    else:
        size_in_kb = size_in_bytes / (2**10)
        return f"{size_in_kb:.2f} KB"

### 5. Run manual tests

In [ ]:
MIN_MESSAGE_SIZE = 1024
MAX_CHUNK_SIZE = 32744  # minus 24 bytes for single chunk header


def get_random_single_chunk_size():
    return random.randint(MIN_MESSAGE_SIZE, MAX_CHUNK_SIZE)


def get_random_multi_chunk_size():
    return random.randint(2 * MAX_CHUNK_SIZE, 3 * MAX_CHUNK_SIZE)

In [ ]:
def test_for_single_chunk_request_and_single_chunk_response():
    request_size = get_random_single_chunk_size()
    response_size = get_random_single_chunk_size()
    total_xfer, total_time, success = send_test_request(request_size, response_size)
    result = "Success" if success else "Failure"
    logging.info(
        f"[{request_size} B ⇅ {response_size} B] "
        f"Transferred {bytes_to_human_readable(total_xfer)} "
        f"in {total_time}s; "
        f"Result: {result}"
    )


test_for_single_chunk_request_and_single_chunk_response()

In [ ]:
def test_for_multi_chunk_request_and_single_chunk_response():
    request_size = get_random_multi_chunk_size()
    response_size = get_random_single_chunk_size()
    total_xfer, total_time, success = send_test_request(request_size, response_size)
    result = "Success" if success else "Failure"
    logging.info(
        f"[{request_size} B ⇅ {response_size} B] "
        f"Transferred {bytes_to_human_readable(total_xfer)} "
        f"in {total_time}s; "
        f"Result: {result}"
    )


test_for_multi_chunk_request_and_single_chunk_response()

In [ ]:
def test_for_single_chunk_request_and_multi_chunk_response():
    request_size = get_random_single_chunk_size()
    response_size = get_random_multi_chunk_size()
    total_xfer, total_time, success = send_test_request(request_size, response_size)
    result = "Success" if success else "Failure"
    logging.info(
        f"[{request_size} B ⇅ {response_size} B] "
        f"Transferred {bytes_to_human_readable(total_xfer)} "
        f"in {total_time}s; "
        f"Result: {result}"
    )


test_for_single_chunk_request_and_multi_chunk_response()

In [ ]:
def test_for_multi_chunk_request_and_multi_chunk_response():
    request_size = get_random_multi_chunk_size()
    response_size = get_random_multi_chunk_size()
    total_xfer, total_time, success = send_test_request(request_size, response_size)
    result = "Success" if success else "Failure"
    logging.info(
        f"[{request_size} B ⇅ {response_size} B] "
        f"Transferred {bytes_to_human_readable(total_xfer)} "
        f"in {total_time}s; "
        f"Result: {result}"
    )


test_for_multi_chunk_request_and_multi_chunk_response()

### 6. Run benchmarks on requests-responses of sizes from 1 KB to 512 MB

In [ ]:
benchmarks = {}

In [ ]:
# Baseline tests (Tests with single chunk messages i.e. 1 KB to 32 KB)
for powers_of_two in range(0, 6):  # Test from 1 KB to 32 KB
    message_size = 2**powers_of_two * 1024
    total_xfer, total_time, success = send_test_request(message_size, message_size)
    if success:
        benchmarks[bytes_to_human_readable(total_xfer)] = total_time
pprint(benchmarks, sort_dicts=False)

In [ ]:
# Tests with smaller messages
for powers_of_two in range(6, 13):  # Test from 64 KB to 4 MB
    message_size = 2**powers_of_two * 1024
    total_xfer, total_time, success = send_test_request(message_size, message_size)
    if success:
        benchmarks[bytes_to_human_readable(total_xfer)] = total_time
pprint(benchmarks, sort_dicts=False)

In [14]:
# Tests with larger messages
for powers_of_two in range(13, 16):  # Test from 8 MB to 32 MB
    message_size = 2**powers_of_two * 1024
    total_xfer, total_time, success = send_test_request(message_size, message_size)
    if success:
        benchmarks[bytes_to_human_readable(total_xfer)] = total_time
pprint(benchmarks, sort_dicts=False)

In [ ]:
# Tests with super large messages
for powers_of_two in range(16, 19):  # Test from 64 MB to 256 MB
    message_size = 2**powers_of_two * 1024
    total_xfer, total_time, success = send_test_request(message_size, message_size)
    if success:
        benchmarks[bytes_to_human_readable(total_xfer)] = total_time
pprint(benchmarks, sort_dicts=False)